In [2]:
import os
import sys
import numpy as np
import pandas as pd
import random

from argparse import ArgumentParser
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn

from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

sys.path.append('../')
from torchlib.datasets.datasets import ATLASDataset 
from torchlib.neuralnet import NeuralNetClassifier
from misc import get_transforms_det

from sklearn import metrics

In [3]:
pathname = os.path.expanduser( '~/.datasets' )
projectnet='../out/netruns'
no_cuda=False
seed=1
gpu=0
batch_size=10
workers=0
num_input_channels=3
parallel=False

print(pathname)
print(projectnet)

/home/fernandez/.datasets
../out/netruns


In [4]:
!ls ../out/netruns/

atlas_baseline_preactresnet18_bcewl_adam_atlas_001


In [5]:
name = 'atlas_baseline_preactresnet18_bcewl_adam_atlas_001'
project = os.path.join(projectnet, name)
model = 'model_best.pth.tar' #'chk000025.pth.tar', model_best
pathmodels = os.path.join(project,'models',model)

print(project)
print(pathmodels)

../out/netruns/atlas_baseline_preactresnet18_bcewl_adam_atlas_001
../out/netruns/atlas_baseline_preactresnet18_bcewl_adam_atlas_001/models/model_best.pth.tar


In [6]:
# neuralnet
network = NeuralNetClassifier(
    patchproject=project,
    nameproject=name,
    no_cuda=no_cuda,
    seed=seed,
    gpu=gpu,
    parallel=parallel,
    )

cudnn.benchmark = True

# load model
if network.load( pathmodels ) is not True:
    assert(False)

Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


=> loading checkpoint '../out/netruns/atlas_baseline_preactresnet18_bcewl_adam_atlas_001/models/model_best.pth.tar'
=> loaded checkpoint for preactresnet18 arch!


In [7]:
pathdata='~/.kaggle/competitions/human-protein-atlas-image-classification'
name_dataset='atlas'
subset='test'

data = ATLASDataset(        
    path=pathdata, 
    train=False,
    folders_images='test', 
    metadata='sample_submission.csv',
    #count=100,
    num_channels=3,
    transform=get_transforms_det( network.size_input ), #get_transforms_aug
    )

dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )

print('Data loader ')
print(len(dataloader))
print(len(data))

Data loader 
1171
11702


In [8]:
Id, Yhat, Y = network.predict( dataloader )

df = pd.DataFrame( np.concatenate((Id[:,np.newaxis], Yhat), axis=1) )
df.to_csv( os.path.join(project , '{}_{}_{}_dp.csv'.format(subset,name,name_dataset)), index=False, encoding='utf-8')        
print('dir: {}'.format(project))
print('DONE!!!')

100%|██████████| 1171/1171 [04:41<00:00,  4.15it/s]


dir: ../out/netruns/atlas_baseline_preactresnet18_bcewl_adam_atlas_001
DONE!!!


In [14]:
df = pd.read_csv( os.path.join(project , '{}_{}_{}_dp.csv'.format(subset, name, name_dataset))  )
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0.014306,3.973154e-07,0.810111,0.243149,1.113648e-11,1.403070e-08,5.526864e-09,4.717684e-07,5.500286e-14,...,6.487467e-09,0.000015,1.035360e-09,0.000006,1.423424e-07,0.000089,1.229151e-11,1.565527e-07,4.480703e-09,1.855383e-12
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0.188160,9.161159e-05,0.000268,0.018457,6.051402e-05,3.521206e-01,2.293978e-01,3.284568e-04,1.486919e-06,...,8.210330e-03,0.001300,1.062604e-03,0.003352,8.179079e-04,0.690412,2.875413e-05,3.671001e-02,3.562972e-02,7.168690e-06
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0.989122,1.159247e-04,0.000898,0.003168,4.198915e-03,1.905118e-01,5.204419e-04,4.018500e-01,3.275845e-07,...,1.043462e-03,0.000351,2.506983e-06,0.007738,3.984306e-07,0.001231,1.668895e-05,9.736225e-01,3.807373e-04,3.476945e-04
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0.557950,1.242832e-02,0.256312,0.002360,1.064062e-02,8.077870e-03,1.050658e-04,2.259287e-02,1.621884e-05,...,2.101554e-02,0.004429,3.386048e-05,0.089971,9.430081e-04,0.022968,6.979299e-05,6.885216e-01,6.807976e-03,1.263778e-03
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0.111725,9.780910e-05,0.007518,0.000122,1.676468e-04,9.079865e-02,7.282246e-04,2.724890e-01,8.146868e-06,...,1.827399e-03,0.000629,1.187180e-04,0.020294,2.287483e-05,0.003374,4.404008e-07,4.609438e-01,1.401578e-04,7.009440e-05


In [10]:
result = df.as_matrix()
iDs = result[:,0]
Yhat = result[:,1:]

print(iDs.shape)
print(Yhat.shape)


(11702,)
(11702, 28)


In [12]:
(Yhat>0.4).sum(axis=0)

array([4738,  384, 1127,  358,  419,  516,  191, 1021,    0,    2,    1,
        271,  151,  125,  412,    0,  111,  131,  213,  333,   32, 1630,
        117,  700,   80, 3706,   70,    1])

In [84]:
pred = [ ' '.join( map(str,np.where( yhat > 0.4 )[0]) ) for yhat in  Yhat]
#code  = ' '.join( map(str, code) )
len(np.where( np.array([1,2,3]) < 0 )[0]) 

0

In [45]:
# submission_filepath = 'submission.csv'
# th=0.5
# #submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0])  if len(np.where( yhat > th )[0])>0 else '0'  ) }  for (Id, yhat) in zip( iDs,Yhat) ]
# submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0]) ) }  for (Id, yhat) in zip( iDs,Yhat) ]
# submission = pd.DataFrame( submission )
# submission.to_csv(submission_filepath, index=False, encoding='utf-8')
# print('SAVE!!!')
# submission


In [16]:
submission_filepath = 'submission.csv'
th=0.4
submission = pd.read_csv('~/.kaggle/competitions/human-protein-atlas-image-classification/sample_submission.csv')

#submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0])  if len(np.where( yhat > th )[0])>0 else '0'  ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission_local = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0]) ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission_local = pd.DataFrame( submission_local )
submission.loc[ submission['Id'] == submission_local['Id']   , 'Predicted'] = submission_local['Predicted']
submission.to_csv(submission_filepath, index=False, encoding='utf-8')
submission



,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,23
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 7 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,4
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 12 23 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 25


In [18]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f submission.csv -m "preactresnet18+bceloss+128x128"

100%|█████████████████████████████████████████| 465k/465k [00:01<00:00, 473kB/s]
Successfully submitted to Human Protein Atlas Image Classification

In [24]:
submission = pd.read_csv('/root/.kaggle/competitions/human-protein-atlas-image-classification/sample_submission.csv')
submission

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,0
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0
